# Notebook Card: Finetuning t5-small model
* * *

This notebook describes how to fine-tune a pre-trained `t5-small` model for the following downstream task:

1. Classification on SST-2
2. Summarization on xsum
3. Question-answering on SQuaD

TODO:

1. Change Model checkpoint
2. Add evaluation after each finetuning section
3. Add QA
4. Change eval_stat to epoch and logging_step to 100
5. Push fine-tuned model to n HF hub

## Pre-requisites

* Setup preferred encoding
* Installation of required libraries

In [ ]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install evaluate transformers[torch] datasets rouge_score bert_score
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 35.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.1 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-

## Importing required *packages*

In [ ]:
import os
import torch
import evaluate
import numpy as np
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, AutoModelForSequenceClassification, DataCollatorWithPadding, Trainer, TrainingArguments
from datasets import load_dataset, DatasetDict

## Required parameters for finetuning

Fill the rendered form below (right side) and then run the cell below

In [ ]:
# Take user inputs

class Arguments:
    pass
args = Arguments()

args.epoch = 5 # @param {type:"number"}
args.model = "t5-small" # @param {type:"string"}
args.batch = 32 # @param {type:"number"}
args.lr = 2e-5 # @param {type:"number"}
args.cuda_device = "0" # @param {type:"string"}
args.max_context_length = 512 # @param {type:"string"}
args.only_evaluate = False # @param {type:"boolean"}
args.random_seed = 1 # @param {type:"number"}
args.test_split = 0.001 # @param {type:"number"}

## Environment Setup

1. Setting seeds for reproducability
2. Configurations for hugging face package
3. GPU/CPU device setup

In [ ]:
# Setting seeds for reproducability
random_seed = args.random_seed
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)

# Configurations for training models
model_checkpoint = args.model # Huggingface model repo name
BATCH_SIZE = args.batch
EPOCH = args.epoch # Number of epochs
LR = args.lr # Learning rate. Just using the one generally used. Needs hyperparameter tuning to improve
MAX_LENGTH = args.max_context_length

# GPU/CPU device setup
CUDA = args.cuda_device
os.environ["CUDA_VISIBLE_DEVICES"] = CUDA
device = torch.device(f"cuda" if torch.cuda.is_available() else "cpu")

# Loading tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

## Fine-tune task (a): Classification


### Load SST-2 dataset and perform 80-20 split

Performing stratify to ensure a balanced dataset

In [ ]:
cls_dataset = load_dataset("sst2", split="train")
cls_dataset = cls_dataset.train_test_split(test_size=args.test_split, seed=1, stratify_by_column="label")
print(cls_dataset)

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 67281
    })
    test: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 68
    })
})


### Perform tokenization for classification

In [ ]:
def cls_preprocess_function(sample):
    """Tokenize the samples
    """
    model_inputs = tokenizer(sample["sentence"], truncation=True)
    model_inputs["labels"] = sample["label"]
    return model_inputs
tokenized_dataset = cls_dataset.map(cls_preprocess_function, batched=True) # This will take 10 min

### Function definition for metrics

1. Accuracy
2. F1
3. Recall
4. Precision

In [ ]:
accuracy = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
recall_metric = evaluate.load('recall')
precision_metric = evaluate.load("precision")

def cls_compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits[0], axis=1)

    # Accuracy
    result = accuracy.compute(predictions=predictions, references=labels)

    # F1
    f1 = f1_metric.compute(predictions=predictions, references=labels)
    result.update(f1)

    # Recall
    recall = recall_metric.compute(predictions=predictions, references=labels)
    result.update(recall)

    # Precision
    precision = precision_metric.compute(predictions=predictions, references=labels)
    result.update(precision)

    return result


### Starting finetuning

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# Mapping of expected ids and labels
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

# Load the model
cls_model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id).to(device)

training_args = TrainingArguments(
    num_train_epochs=EPOCH,
    output_dir="data/sst2",
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    weight_decay=0.01,
    save_total_limit=1, # Number of models that need to be saved
    warmup_steps=500,
    fp16=True,
    report_to = "tensorboard"
    # push_to_hub=True,
)

trainer = Trainer(
    model=cls_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=cls_compute_metrics
)


if not args.only_evaluate:
    trainer.train()
else:
    eval_results = trainer.evaluate()
    print(eval_results)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy,F1,Recall,Precision
10,0.699700,0.685005,0.602941,0.630137,0.605263,0.657143
20,0.725400,0.684951,0.573529,0.613333,0.605263,0.621622
30,0.689100,0.684883,0.602941,0.630137,0.605263,0.657143
40,0.705200,0.684800,0.602941,0.630137,0.605263,0.657143
50,0.710500,0.684269,0.558824,0.605263,0.605263,0.605263
60,0.703900,0.685001,0.544118,0.586667,0.578947,0.594595
70,0.698800,0.684175,0.544118,0.586667,0.578947,0.594595
80,0.707900,0.682362,0.544118,0.586667,0.578947,0.594595
90,0.706800,0.681766,0.544118,0.586667,0.578947,0.594595
100,0.697000,0.680574,0.617647,0.675000,0.710526,0.642857


KeyboardInterrupt: ignored

## Fine-tune task (b): Summarization


### Load xsum dataset and perform 80-20 split

In [ ]:
summarize_dataset = load_dataset("EdinburghNLP/xsum", split="train")
summarize_dataset = summarize_dataset.train_test_split(test_size=args.test_split, seed=1)
print(summarize_dataset)

### Perform tokenization for summarization

For Models like T5 it is better to add the commands like `prefix` variable to have better performance as they are designed for multiple NLP Tasks [reference](https://huggingface.co/docs/transformers/tasks/summarization)

In [ ]:
prefix = "summarize: "

def preprocess_function(sample):
    inputs = [prefix + doc for doc in sample["document"]] # Document is the input to the model in the encoder part
    model_inputs = tokenizer(inputs, max_length=MAX_LENGTH, truncation=True)

    labels = tokenizer(text_target=sample["summary"], max_length=MAX_LENGTH, truncation=True) # Summary of the document that is the output (label) for the model

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
tokenized_dataset = summarize_dataset.map(preprocess_function, batched=True) # Takes around 10min

### Function definition for metrics

1. Rogue - Computes ROUGE-1, ROUGE-2, ROUGE-L, ROUGE-L
2. BLEU Score
3. BERTScore
4. Meteor

In [ ]:
# Evaluation metric: Rogue
rouge = evaluate.load("rouge") # A metric in which the value is between 0 and 1; higher the better
bleu = evaluate.load("bleu")
bertscore = evaluate.load("bertscore")
meteor = evaluate.load('meteor')

# Compute metric function- executed after every epoch
def summarize_compute_metrics(eval_pred):
    predictions, labels = eval_pred # Contains both predicted and true labels (labels)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True) # Int Token to String
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id) # Removing padding
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True) # Int Token to String

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True) # Computing the metric

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    # Bleu Score
    result["bleu"] = bleu.compute(predictions=decoded_preds, references=decoded_labels)['bleu']

    # BERTScore
    bert_score = bertscore.compute(predictions=decoded_preds, references=decoded_labels, model_type="distilbert-base-uncased")
    result.update(bert_score)
    # Taking mean of lists
    result["precision"] = np.mean(result["precision"])
    result["recall"] = np.mean(result["recall"])
    result["f1"] = np.mean(result["f1"])

    # METEOR Score
    meteor_score = meteor.compute(predictions=decoded_preds, references=decoded_labels)
    result.update(meteor_score)

    del result['hashcode']

    return result

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


### Starting finetuning

In [ ]:
# Load model
summarize_model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to(device)
# load Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=summarize_model, label_pad_token_id=tokenizer.pad_token_id)
training_args = Seq2SeqTrainingArguments(
    num_train_epochs=EPOCH,
    output_dir="data/xsum",
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    weight_decay=0.01,
    save_total_limit=1, # Number of models that need to be saved
    predict_with_generate=True,
    warmup_steps=500,
    fp16=True,
    report_to = "tensorboard"
    # push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=summarize_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=summarize_compute_metrics,
)


if not args.only_evaluate:
    trainer.train()
else:
    eval_results = trainer.evaluate()
    print(eval_results)

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Bleu,Precision,Recall,F1,Meteor
10,1.700100,1.477579,0.256309,0.063157,0.200093,0.199796,18.775610,0.024595,0.782969,0.757413,0.769703,0.161475


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


KeyboardInterrupt: ignored

## Fine-tune task (c): Question Answering